In [1]:
# import all you need
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import curve_fit
from helper_files.plotting import plot_lines, fitting_plot
from helper_files.gaussian_fitting import gaussian, n_gaussians, fit_n_peaks_to_gaussian
from helper_files.read_data import read_data

In [2]:
# this will load the helper modules each time you make changes to them
%load_ext autoreload
%autoreload 2 # or 1?????

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# read the file

# set the path to the file, or just use the example file
filepath = 'Example_data_EDS_GaAs_30kV_NTNU_NanoLab.emsa'

# open the file and find out where the data starts and stops.
# in the example data the data starts after "#SPECTRUM    : Spectral Data Starts Here",
# and ends with "#ENDOFDATA   : "

# take note of the line endings,
# in the example data the line endings are '\n'

# take note of what the data is separated by, 
# in the example data it is separated by a comma and a space ", "

# adapt these values to your data
start_string = "#SPECTRUM    : Spectral Data Starts Here"
stop_string = "#ENDOFDATA   : "
line_endings = '\n'
delimiter = ', '


# from the helper file read_data.py, using the function read_data
data = read_data(filepath, start_string, stop_string, delimiter, line_endings)

# normalize counts to 1
y_raw = data[1]/data[1].max()
x_raw = np.linspace(0, len(y_raw), len(y_raw))

# for some reason the fitting only works in the x values are integers.
# since the goal is to calibrate the channels, we can just use the channel numbers as x values for now.


Read 2048 data points from Example_data_EDS_GaAs_30kV_NTNU_NanoLab.emsa
First entry: [-0.2, 0.0]
Last entry: [20.27, 52.0]


In [4]:
# now we find two peaks in the data, and we want to fit a gaussian to each of them,
# and we need to know what the value of the peaks in theory should be.
# in the example data we can use Ga_Ka=9.2517 keV and Ga_La=1.098 keV,
# or we can use As_Ka=10.5336 keV and As_La=1.2819 keV


# peak_guesses = [9.2517, 10.5336]

# see file 03_gaussian_fitting.py

# def gaussian(x, amp, mu, sigma):
# the function gaussian defines a gaussian function.

# def n_gaussians(x, *args):
# since the gaussians could potentially partially overlap, 
# like Ga_Kb=10.2642 and As=10.5436 in the example data,
# we need to define a function that returns the sum of n gaussians.
# this allows to fit multiple gaussians to the data at once,
# but two is enough for our purposes.

# def fit_n_peaks_to_gaussian(x, raw_y, guessed_peaks, guessed_std=1, guessed_amp=1,):
# now we need a functions which fits peak guesses to gaussian curves.
# we will use the scipy.optimize.curve_fit function for this. More info in the function.







In [5]:
plot_lines(data[0], data[1], title='Plot of the raw data', xaxis_title='uncalibrated keV', yaxis_title='Counts')

In [6]:
plot_lines(x_raw, y_raw, title='Plot of the data we will work with', xaxis_title='channels', yaxis_title='Normalized counts')

In [13]:
# originally i tried this, but that did not work. 
# The fitting stops when I use keV at x and the counts at y.
# I think it is because the x values are not integers.
# peak_guesses = [0.02, 0.27, 0.97, 1.1, 1.29, 1.75, 9.2517, 10.24, 10.5336, 11.75] 
# fit_vals = fit_n_peaks_to_gaussian(data[0], data[1], peak_guesses)


# the peak guesses are the approximate values of the peaks in the plot above
guesses_peaks_Ga30 = [22, 46, 117, 130, 149, 195, 944, 1045, 1072, 1191]

# running the gaussian fitting function, see helper_files/gaussian_fitting.py
fit_vals = fit_n_peaks_to_gaussian(x_raw, y_raw, guesses_peaks_Ga30)

# fit_vals[0]

In [14]:
# advanced plot using the following function, see helper_files/plotting.py
#fitting_plot(x, y_raw, y_fit=None, vlines=None, fit_vals=None, fig=None, start=0, stop=2048, 
# title="Fitting plot", xaxis_title="Channel number [~10eV]", yaxis_title="Relative intensity [a.u.]", )

y_fit = n_gaussians(x_raw, *fit_vals[0])
peaks_fitted = fit_vals[0][1::3]  # every third value is the peak value

fig = fitting_plot(x_raw, y_raw, y_fit=y_fit, vlines=peaks_fitted)  # add fit_vals[0] to see the fit values
fig.show()

In [9]:
# # since we are clever, we know that we can plot the fitted data on top of the raw x values,
# # but the fitted peaks are not at the correct values.
# fig = fitting_plot(data[0], y_raw, y_fit=y_fit, xaxis_title='uncalibrated keV', title='Plotting the raw and fit, on the raw x values')
# fig.show()

In [10]:
# now we want to plot the results above on the raw x values,
# but we need to know the calibration factor.
# that is, we

# linear mapping of the channels to keV
kev0 = data[0][0]
kev1 = data[0][-1]
channel0 = x_raw[0]
channel1 = x_raw[-1]

# the slope of the linear mapping
slope = (kev1-kev0)/(channel1-channel0)
print(f'linmap: {kev0} + {slope:.3e}*channel')

peaks_kev = [slope*(peak-channel0)+kev0 for peak in peaks_fitted]


linmap: -0.2 + 9.995e-03*channel


In [15]:
fig = fitting_plot(data[0], y_raw, y_fit=y_fit, vlines=peaks_kev, xaxis_title='uncalibrated keV', title='Plotting the raw and fit, on the raw x values')
fig.show()

# to save the figure, uncomment the following line
# fig.write_image('Example_data_plot_uncalibrated.html')

In [12]:
# issue now: we have negative vaules, and i dunno what to do.

# assuming that Ga_Ka=9.2517, we need some 